In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)



import tensorflow as tf



# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/fake-news/submit.csv
/kaggle/input/fake-news/train.csv
/kaggle/input/fake-news/test.csv


## Datasets Loading.....

In [2]:
df = pd.read_csv('/kaggle/input/fake-news/train.csv')

In [3]:
test_df = pd.read_csv('/kaggle/input/fake-news/test.csv')

In [4]:
submission = pd.read_csv('/kaggle/input/fake-news/submit.csv')

In [5]:


main_df = pd.concat([df.drop(['label'],axis=1),test_df],axis=0)

In [6]:
main_df.shape

(26000, 4)

## Missing Data

In [7]:
main_df.title.head(5)

0    House Dem Aide: We Didn’t Even See Comey’s Let...
1    FLYNN: Hillary Clinton, Big Woman on Campus - ...
2                    Why the Truth Might Get You Fired
3    15 Civilians Killed In Single US Airstrike Hav...
4    Iranian woman jailed for fictional unpublished...
Name: title, dtype: object

In [8]:
main_df.text.head(5)

0    House Dem Aide: We Didn’t Even See Comey’s Let...
1    Ever get the feeling your life circles the rou...
2    Why the Truth Might Get You Fired October 29, ...
3    Videos 15 Civilians Killed In Single US Airstr...
4    Print \nAn Iranian woman has been sentenced to...
Name: text, dtype: object

In [9]:
main_df.title=main_df.title.fillna(main_df['text'])

In [10]:






main_df.isnull().sum()

id           0
title        0
author    2460
text        46
dtype: int64

In [11]:
main_df.text=main_df.text.fillna(main_df.title)

In [12]:
main_df.isnull().sum()

id           0
title        0
author    2460
text         0
dtype: int64

In [13]:
main_df[main_df.author.isnull()]

,id,title,author,text
6,6,Life: Life Of Luxury: Elton John’s 6 Favorite ...,NaN,Ever wonder how Britain’s most iconic pop pian...
8,8,Excerpts From a Draft Script for Donald Trump’...,NaN,Donald J. Trump is scheduled to make a highly ...
20,20,News: Hope For The GOP: A Nude Paul Ryan Has J...,NaN,Email \nSince Donald Trump entered the electio...
23,23,Massachusetts Cop’s Wife Busted for Pinning Fa...,NaN,Massachusetts Cop’s Wife Busted for Pinning Fa...
31,31,Israel is Becoming Pivotal to China’s Mid-East...,NaN,Country: Israel While China is silently playin...
...,...,...,...,...
5152,25952,Flotilla of Russian warships sails into the Me...,NaN,"09:01 EST, 12 November 2016 | Updated: 18:54 E..."
5163,25963,TRUMP RECEIVED BY MASSIVE CROWD IN ALBUQUERQUE...,NaN,Home / News / TRUMP RECEIVED BY MASSIVE CROWD ...
5170,25970,Reinventing Democracy in America Starts by Vot...,NaN,Reinventing Democracy in America Starts by Vot...
5172,25972,The Postmodern Stalinism of the Western Media ...,NaN,Originally appeared at Chronicles Magazine \nI...


In [14]:
main_df.author = main_df.author.fillna('unknown')

In [15]:
main_df.isnull().sum()

id        0
title     0
author    0
text      0
dtype: int64

In [16]:
# We will be only using title and author name for prediction
# Creating new coolumn total concatenating title and author
main_df['total'] = main_df['title']+' '+main_df['author']
#test['total']=test['title']+' '+test['author']

In [17]:
X= main_df
y = df['label']

## Pre_processing...

In [18]:
from tensorflow.keras.layers import Embedding ,LSTM , Dense,Dropout
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.models import Sequential

In [19]:
voc_size= 5000
messages = X.copy()
messages.reset_index(inplace=True)

In [20]:
import nltk
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer


In [21]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [22]:
#Applying stemming and some preprocessing
ps = PorterStemmer()
corpus = []
for i in range(0,len(messages)):
   
    review = re.sub('[^a-zA-Z]',' ',messages['total'][i])
    review =review.lower()
    review = review.split()
    review = [ps.stem(word)for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [23]:
corpus[0]

'hous dem aid even see comey letter jason chaffetz tweet darrel lucu'

In [24]:
one_represent = [one_hot(words,voc_size) for words in corpus]

In [25]:
one_represent[0]

[4347, 2809, 2972, 3311, 2492, 2048, 3239, 4538, 2383, 1765, 1906, 737]

In [26]:
sent_length = 20
embedded_docs = pad_sequences(one_represent,padding='pre',maxlen=sent_length)
print(embedded_docs[0])

[   0    0    0    0    0    0    0    0 4347 2809 2972 3311 2492 2048
 3239 4538 2383 1765 1906  737]


## Model creating...

embedding_vector_features = 40
model = Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [27]:
#We have used embedding layers with LSTM
model = Sequential()
model.add(Embedding(voc_size,40,input_length=25))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(64,activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 25, 40)            200000    
_________________________________________________________________
dropout (Dropout)            (None, 25, 40)            0         
_________________________________________________________________
lstm (LSTM)                  (None, 100)               56400     
_________________________________________________________________
dropout_1 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense (Dense)                (None, 64)                6464      
_________________________________________________________________
dropout_2 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 6

In [28]:
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 25, 40)            200000    
_________________________________________________________________
dropout (Dropout)            (None, 25, 40)            0         
_________________________________________________________________
lstm (LSTM)                  (None, 100)               56400     
_________________________________________________________________
dropout_1 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense (Dense)                (None, 64)                6464      
_________________________________________________________________
dropout_2 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 6

In [29]:
len(embedded_docs),y.shape


(26000, (20800,))

In [30]:
X_final=np.array(embedded_docs)
X_train_final  = X_final[:20800]

X_test_final   = X_final[20800:]
y_final = np.array(y)
X_test_final.shape

(5200, 20)

In [31]:
from sklearn.model_selection import train_test_split

In [32]:
X_train,X_test,y_train,y_test =train_test_split(X_train_final ,y_final ,test_size=0.33,random_state=42)


In [33]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=20,batch_size=64)

Epoch 1/20
218/218 [==============================] - 9s 31ms/step - loss: 0.3909 - accuracy: 0.7932 - val_loss: 0.0303 - val_accuracy: 0.9904
Epoch 2/20
218/218 [==============================] - 6s 27ms/step - loss: 0.0173 - accuracy: 0.9953 - val_loss: 0.0254 - val_accuracy: 0.9932
Epoch 3/20
218/218 [==============================] - 6s 29ms/step - loss: 0.0049 - accuracy: 0.9986 - val_loss: 0.0241 - val_accuracy: 0.9934
Epoch 4/20
218/218 [==============================] - 6s 27ms/step - loss: 0.0034 - accuracy: 0.9991 - val_loss: 0.0302 - val_accuracy: 0.9927
Epoch 5/20
218/218 [==============================] - 6s 29ms/step - loss: 0.0024 - accuracy: 0.9995 - val_loss: 0.0375 - val_accuracy: 0.9918
Epoch 6/20
218/218 [==============================] - 6s 27ms/step - loss: 0.0022 - accuracy: 0.9994 - val_loss: 0.0361 - val_accuracy: 0.9929
Epoch 7/20
218/218 [==============================] - 6s 28ms/step - loss: 4.6854e-04 - accuracy: 0.9999 - val_loss: 0.0426 - val_accuracy: 0.

## predictions...

In [34]:
y_pred = model.predict_classes(X_test_final)


/opt/conda/lib/python3.7/site-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


In [35]:

y_pred.shape,test_df.id.shape

((5200, 1), (5200,))

In [36]:
submission.head()

,id,label
0,20800,0
1,20801,1
2,20802,0
3,20803,1
4,20804,1


In [37]:
y_pred=y_pred.reshape(5200)
type(y_pred)


numpy.ndarray

In [38]:
submission = pd.DataFrame({'id':test_df.id,'label':y_pred})

In [39]:
submission.head()

,id,label
0,20800,0
1,20801,1
2,20802,1
3,20803,0
4,20804,1


In [40]:
submission.shape

(5200, 2)

In [41]:
submission.to_csv('LSMT_model_work.csv',index=False)